In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.tokenize import word_tokenize
import nltk

In [101]:
train = pd.read_json('mts/snli_1.0_train.jsonl', lines=True)

In [102]:
train.head(3)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...


In [103]:
print(train['sentence1'][0],'\n')
print(train['sentence1_binary_parse'][0],'\n')
print(train['sentence1_parse'][0])

A person on a horse jumps over a broken down airplane. 

( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ) 

(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN on) (NP (DT a) (NN horse)))) (VP (VBZ jumps) (PP (IN over) (NP (DT a) (JJ broken) (JJ down) (NN airplane)))) (. .)))


In [104]:
train.drop(['captionID', 'pairID', 'annotator_labels'], axis=1, inplace=True)

In [105]:
def choose_balanced_subset(data, n):
    '''Функция выбирает n элементов из data, формируя сбалансированную выборку. Аппендит, перемешивает и сбрасывает индексы'''
    
    
    neutral_labels = data.query('gold_label == "neutral"').iloc[:n]
    contradiction_labels = data.query('gold_label == "contradiction"').iloc[:n]
    entailment_labels = data.query('gold_label == "entailment"').iloc[:n]
    
    return entailment_labels.append(contradiction_labels.append(neutral_labels)).sample(frac=1).reset_index(drop=True)

In [107]:
result = choose_balanced_subset(train, 100)

In [108]:
result.head(3)

,gold_label,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,neutral,"A woman wearing all white and eating, walks ne...",( ( ( A woman ) ( ( ( wearing ( all white ) ) ...,(ROOT (S (NP (NP (DT A) (NN woman)) (VP (VP (V...,A married couple is walking next to each other.,( ( A ( married couple ) ) ( ( is ( ( walking ...,(ROOT (S (NP (DT A) (JJ married) (NN couple)) ...
1,neutral,The school is having a special event in order ...,( ( The school ) ( ( is ( ( having ( ( a ( spe...,(ROOT (S (NP (DT The) (NN school)) (VP (VBZ is...,A high school is hosting an event.,( ( A ( high school ) ) ( ( is ( hosting ( an ...,(ROOT (S (NP (DT A) (JJ high) (NN school)) (VP...
2,contradiction,"A woman wearing all white and eating, walks ne...",( ( ( A woman ) ( ( ( wearing ( all white ) ) ...,(ROOT (S (NP (NP (DT A) (NN woman)) (VP (VP (V...,A married couple is sleeping.,( ( A ( married couple ) ) ( ( is sleeping ) ....,(ROOT (S (NP (DT A) (JJ married) (NN couple)) ...


In [109]:
result.drop(['sentence1_binary_parse','sentence2_binary_parse','sentence1_parse', 'sentence2_parse' ], axis=1, inplace=True)

In [150]:
result['tokenized_sentence1'] = result.apply(lambda row: str(word_tokenize(row['sentence1']))[1:-1], axis=1) 
# slice is to drop brackets

In [151]:
result['tokenized_sentence1']

0      'A', 'woman', 'wearing', 'all', 'white', 'and'...
1      'The', 'school', 'is', 'having', 'a', 'special...
2      'A', 'woman', 'wearing', 'all', 'white', 'and'...
3      'a', 'skateboarder', 'skates', 'in', 'the', 'p...
4      'A', 'foreign', 'family', 'is', 'walking', 'al...
                             ...                        
295    'A', 'man', 'with', 'a', 'red', 'shirt', 'is',...
296    'There', 'is', 'a', 'woman', 'holding', 'a', '...
297    'Bicyclists', 'waiting', 'at', 'an', 'intersec...
298    'A', 'foreign', 'family', 'is', 'walking', 'al...
299    'The', 'school', 'is', 'having', 'a', 'special...
Name: tokenized_sentence1, Length: 300, dtype: object

In [11]:
train = train.iloc[:7000]

In [12]:
test = train.iloc[6000:7000]

In [4]:
from gensim.models import Word2Vec

In [13]:
train.head(3)

,annotator_labels,gold_label,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],neutral,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],contradiction,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],entailment,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...


In [ ]:
neutral_label = 3002

#### хитмапа в конце